# Building Graph-based Applications

This notebook shows a way to use `ipyradiant` widgets to build a simple tab-based
application.

In [ ]:
import traitlets as T

import ipywidgets as W
from ipyradiant import CytoscapeVisualizer, FileManager, PathLoader, QueryWidget
from rdflib import BNode, Graph

## The App Class

This class-based widget captures both the overall layout, as well its individual
components. Values are
[linked](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#Linking-Widgets)
to create new behaviors.

In [ ]:
class RadiantTabs(W.Tab):
    graph = T.Instance(Graph, allow_none=True)
    file_manager = T.Instance(FileManager)
    query = T.Instance(QueryWidget)
    vis = T.Instance(CytoscapeVisualizer)
    log = W.Output()

    def __init__(self, graph: Graph = None, *args, **kwargs):
        if graph is not None:
            kwargs["graph"] = graph
        if "layout" not in kwargs:
            kwargs["layout"] = {
                "min_height": "400px",
                "flex": "1",
            }
        super().__init__(*args, **kwargs)
        T.link((self.file_manager, "graph"), (self, "graph"))
        T.link((self, "graph"), (self.query, "graph"))
        T.link((self, "graph"), (self.vis, "graph"))

        self.children = [self.file_manager, self.query, self.vis]
        self.set_title(0, "Load")
        self.set_title(1, "Query")
        self.set_title(2, "Visualize")

    @T.default("graph")
    def make_default_graph(self):
        return Graph()

    @T.default("file_manager")
    def make_default_load_widget(self):
        return FileManager(loader=PathLoader(path="data"))

    @T.default("query")
    def make_default_query_widget(self):
        return QueryWidget()

    @T.default("vis")
    def make_vis_widget(self):
        return CytoscapeVisualizer()

## Showing the App

In [ ]:
tabs = RadiantTabs()
W.VBox([tabs, tabs.query.log])